In [1]:
# SKT Brain github 주소는 다음과 같습니다. https://github.com/SKTBrain/KoBERT

# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3 # 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
# !pip install torch

# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

# kobert 
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

In [2]:
# GPU 사용 시
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Bert모델, Voca 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# cuda device 확인
torch.cuda. empty_cache()

#GPU 디바이스의 갯수
print ('Available devices ', torch.cuda.device_count())
#현재 셋업 되어있는 GPU 넘버
print ('Current cuda device ', torch.cuda.current_device())
#GPU 디바이스의 이름
print(torch.cuda.get_device_name(device))

using cached model
using cached model
Available devices  1
Current cuda device  0
Tesla P100-PCIE-16GB


In [3]:
# kobert 입력 데이터로 만들기

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
    
    
# 토큰화
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)    
    
    
# Kobert 학습모델 만들기

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      

using cached model


In [4]:
# path 설정
PATH = '../'

In [5]:
model = torch.load(PATH + 'model11.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict11.pt'))  # state_dict를 불러 온 후, 모델에 저장



<All keys matched successfully>

In [6]:
# Setting parameters 파라미터 세팅
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 9 # 훈련 반복횟수
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [7]:
test=pd.read_csv(PATH + 'news_sentiment/test77.csv')
print(f'test.shape:{test.shape}')
test.head()

test.shape:(131598, 2)


,Unnamed: 0,TitleArticleToken
0,0,라파엘 제약 단계 평가 사전 계획 긍정 중간 효용 분석 발표 로이터 라파엘 홀딩스 ...
1,1,업데이트 미국 징후 로이터 주식회사 미국 위암 대하 표시 대하 업데이트 종양 발현 ...
2,2,주식 상장 가격 공개 로이터 주식 상장 달러 주당 달러 가격 대비 공개
3,3,비철 금속 순이익 증가 예상 로이터 통링 금속 그룹 순이익 위안 전년 동기 대비 증...
4,4,일자 지분 로이터 현재 지분 보고 주식 매입 저평가 주식 매입


In [8]:
title = list(test['TitleArticleToken'])
new_data2 = pd.DataFrame(data=title, columns=['data'])



new_data2.head()

,data
0,라파엘 제약 단계 평가 사전 계획 긍정 중간 효용 분석 발표 로이터 라파엘 홀딩스 ...
1,업데이트 미국 징후 로이터 주식회사 미국 위암 대하 표시 대하 업데이트 종양 발현 ...
2,주식 상장 가격 공개 로이터 주식 상장 달러 주당 달러 가격 대비 공개
3,비철 금속 순이익 증가 예상 로이터 통링 금속 그룹 순이익 위안 전년 동기 대비 증...
4,일자 지분 로이터 현재 지분 보고 주식 매입 저평가 주식 매입


In [9]:
coment = title
n = 0
commnetslist = [] # 텍스트 데이터를 담을 리스트
emo_list = [] # 감성 값을 담을 리스트
for c in coment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정

pdData = pd.DataFrame( commnetslist, columns = [['data', 'label']] )
pdData = pdData.values

In [10]:
test_set = BERTDataset(pdData, 0, 1, tok, batch_size, True, False) 
len(test_set)

131598

In [11]:
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
len(test_input)

131598

In [12]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
    # ott = out -> list로 바꿔주고
    ott = out.tolist()
    emo_list.append(ott[0].index(max(ott[0])))

/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/131598 [00:00<?, ?it/s]

In [13]:
len(emo_list)

131598

In [14]:
bert=pd.read_csv(PATH + 'news_sentiment/bert_kospi.csv')

In [15]:
bert['senti']=emo_list

In [16]:
bert

,Unnamed: 0,TitleArticleToken,label,senti
0,0,라파엘 제약 단계 평가 사전 계획 긍정 중간 효용 분석 발표 로이터 라파엘 홀딩스 ...,173,0
1,1,업데이트 미국 징후 로이터 주식회사 미국 위암 대하 표시 대하 업데이트 종양 발현 ...,173,1
2,2,주식 상장 가격 공개 로이터 주식 상장 달러 주당 달러 가격 대비 공개,173,0
3,3,비철 금속 순이익 증가 예상 로이터 통링 금속 그룹 순이익 위안 전년 동기 대비 증...,150,0
4,4,일자 지분 로이터 현재 지분 보고 주식 매입 저평가 주식 매입,152,0
...,...,...,...,...
131593,131593,다음 이상 찜통 열대야 계속 전국 폭염 뉴시스 다음 전국 대부분 지역 기온 이상 오...,74,0
131594,131594,올댓차이 중국 타이산 원전 운전 중단 파손 연료봉 교환타이산 최근 방사 물질 유출 ...,112,0
131595,131595,올댓차이 중국 타이산 원전 운전 중단 파손 연료봉 교환타이산 최근 방사 물질 유출 ...,112,0
131596,131596,스트레스 고민 말끔히 벨레다 아로마 샤워 시리즈아로마 샤워 에너지 하모니 릴렉스 러...,142,1


In [17]:
bert.to_csv('bert_result.csv', index=False)

In [8]:
# 감성값 예측하는 함수 만들기

import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
    commnetslist = [] # 텍스트 데이터를 담을 리스트
    emo_list = [] # 감성 값을 담을 리스트
    for c in comment: # 모든 댓글
        commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정

    pdData = pd.DataFrame( commnetslist, columns = [['뉴스', '감성']] )
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length 
        # 이때, out이 예측 결과 리스트
        
        out = model(token_ids, valid_length, segment_ids)
        print(out)
        # e는 2가지 실수 값으로 구성된 리스트
        # 0번 인덱스가 더 크면 부정, 긍정은 반대
        for e in out:
            if e[0]>e[1]: # 부정
                value = 0
                emo_list.append("부정")
                print('부정')
            else: #긍정
                value = 1
                emo_list.append("긍정")
                print('긍정')
                

    return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환

# input : 텍스트 데이터 리스트 외 KoBERT 설정 파라미터들
# output : 입력한 텍스트 데이터 리스트와 1대 1 매칭 되는 감성 값 리스트

In [10]:
# 뉴스기사 테스트 함수
def news():
    
    comment = []
    comment.append(input("원하는 기사를 입력하세요\n\n"))

    for c in comment:
        print(f'\n기사 : {c}\n')
        
    return getSentimentValue(comment, tok, max_len, batch_size, device)

news()

원하는 기사를 입력하세요

특히 이베스트투자증권은 목표주가를 기존 131만원에서 93만9000원으로 내리고 투자의견도 '매수'에서 '홀드(중립)'로 하향 조정해 눈길을 끌었다. 이베스트투자증권은 지주사 할인율 30%를 적용했다.  이안나 이베스트투자증권 연구원은 "하반기 주도 섹터로 EV(전기자동차) 배터리를 주목하지만 LG에너지솔루션 상장 시기가 다가오고 있으며 SK이노베이션 물적분할 가능성까지 고려할때 배터리 셀 기업 주가는 당분간 지지부진할 것으로 예상된다"고 했다.  앞서 5월 글로벌 투자은행 크레디트스위스(CS)도 LG에너지솔루션 IPO를 언급하면서 LG화학에 대한 투자 의견을 아웃퍼폼(시장 수익률 상회)→언더퍼폼(시장 수익률 하회), 목표 주가를 130만→68만 원으로 47.7% 낮춘 바 있다.  실제 LG화학 주가는 지난 5월 이후 80만원대 초반선에서 크게 벗어나지 못하고 있다. 시가총액도 58조원 내외에 갇혀있다.  하지만 중장기적인 관점에서 반등 여지도 분명히 있다고 본다. 전창현 IBK투자증권 연구원은 "최근 높은 업황 기대감에도 불구하고 부진한 주가를 시형했지만 상장(9~11월)을 기점으로 예고된 불확실성이 해소되며 배터리 사업부의 적정가치를 재평가받을 수 있다"고 했다.  또 전날 LG화학이 LG전자 분리막 코팅사업을 5250억원에 양수한다고 공시한 것에 대한 평가가 긍정적이다. 전유진 하이투자증권 연구원은 "양극재/분리막 등 주요 소재에 대해 LG화학 첨단소재부문이 개발-생산 전 과정을 통합 및 전담해 경쟁력을 강화하겠다는 뜻으로 해석된다"고 했다.  전 연구원은 "향후 전지소재쪽으로 성장 발판을 본격적으로 확보하려는 움직임이 이미 시작된 것으로 신규 동력원이 부재하다는 시장 우려가 과도한 기우임을 보여줬다"고 평가했다.

기사 : 특히 이베스트투자증권은 목표주가를 기존 131만원에서 93만9000원으로 내리고 투자의견도 '매수'에서 '홀드(중립)'로 하향 조정해 눈길을 끌었다. 이베스트투자증권은 지주사 할인율 30%를 적용했다.  이

['부정']